In [1]:
import numpy as np
from sklearn.datasets import make_moons
import tensorflow as tf

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argum

In [21]:
moons = make_moons()

train, labels = moons[0], moons[1].reshape((-1, 1))
m, n = train.shape
train_std = (train - np.mean(train, axis=0))/np.std(train, axis=0)
norm_bias = np.c_[np.ones((m, 1)), train_std]

x_train = norm_bias[:50]
y_train = labels[:50]

x_test = norm_bias[50:]
y_test = labels[50:]

n_epoch = 1000
learning_rate = 0.01

In [3]:
# X = tf.constant(norm_bias, dtype=tf.float32, name="X")
# y = tf.constant(labels.reshape(-1, 1), dtype=tf.float32, name="y")
# theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
# prob = tf.sigmoid(-tf.matmul(X, theta))
# pred = tf.round(prob)

# logloss = -tf.reduce_mean(pred * tf.log(prob) + (1-pred) * tf.log(1-prob))
# gradient = 1/m * tf.matmul(tf.transpose(X), prob - pred)
# training_op = tf.assign(theta, theta-learning_rate * gradient)

# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#     sess.run(init)
    
#     for epoch in range(n_epoch):
#         if epoch % 100 == 0:
#             print("Epoch:", epoch, "LogLoss = ", logloss.eval())
#         sess.run(training_op)
        
#         final_theta = theta.eval()

In [4]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [5]:
X = tf.placeholder(tf.float32, [None, n+1])
y = tf.placeholder(tf.float32, [None, 1])
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
prob = tf.sigmoid(-tf.matmul(X, theta))
pred = tf.round(prob)

logloss = -tf.reduce_mean(pred * tf.log(prob) + (1-pred) * tf.log(1-prob))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(logloss)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epoch):
        rand_index = np.random.choice(100)
        rand_x = [norm_bias[rand_index]]
        rand_y = [labels[rand_index]]
        if epoch % 100 == 0:
            print("Epoch:", epoch, "LogLoss = ", sess.run(logloss, feed_dict={X: rand_x, y: rand_y}
                                                         ))
        sess.run(training_op, feed_dict={X: rand_x, y: rand_y})
        
        final_theta = theta.eval()

Epoch: 0 LogLoss =  0.5354365
Epoch: 100 LogLoss =  0.39628494
Epoch: 200 LogLoss =  0.10696957
Epoch: 300 LogLoss =  0.3750423
Epoch: 400 LogLoss =  0.09366602
Epoch: 500 LogLoss =  0.3283423
Epoch: 600 LogLoss =  0.11879829
Epoch: 700 LogLoss =  0.11819396
Epoch: 800 LogLoss =  0.105071306
Epoch: 900 LogLoss =  0.14723867


In [6]:
final_theta

array([[-1.8430817 ],
       [-1.1393532 ],
       [-0.26066798]], dtype=float32)

In [7]:
probs = sigmoid(np.matmul(norm_bias, final_theta))

In [8]:
def predict_class(probs, cutoff):
    return np.array([1 if i > cutoff else 0 for i in probs])

In [9]:
preds = predict_class(probs, 0.5)

In [10]:
preds

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
np.sum([1 for i in preds.reshape((-1, 1)) - labels if i != 0])/preds.shape[0]

0.56

# Sklearn Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression 

In [13]:
clf = LogisticRegression().fit(norm_bias[:50, :], labels[:50].reshape(-1, ))

In [14]:
sk_preds = clf.predict(norm_bias[50:, :]).reshape((-1, 1))

In [15]:
np.sum(sk_preds - labels[50:])/sk_preds.shape[0]

0.06